# Assignment 3 (Concise, without reading data from external source)


$$ \sum_{t=1,...T}\sum_{j=1,..,4}h_{t,j}inv_{j,t}+\sum_{t=1,...T}\sum_{i=A,...G,j=1,..,4} (prod\_cost_{t,i,j}+trans\_cost_{t,i,j})x_{t, i,j}\\
s.t. \\
Demand: Inv_{t-1,j}+\sum_{i=A,...,G}x_{t,i,j} - Demand_{t,j}=Inv_{t,j}, j=1,2,3,4, t=1,...,T\\
Capacity: \sum_{j=1,...,4}proc\_time_{t,i,j}x_{t,i,j}\le Capacity_{t,i} i=A,...,G, t=1,...,T\\
A1: x_{t,A,1}=0 \\
D1: x_{t,D,1}=0 \\
$$

## Import PuLP modeler functions

In [1]:
from pulp import *
#import pandas as pd
#import numpy as np

## Original Problem

In [2]:
probA=LpProblem("Problem A",LpMinimize)
# creates a list of all cities
factory=['A','B','C','D','E','F','G']
# Creates a list of tuples containing all the possible routes for transport
yarn = ['1','2','3','4']
T=[0,1,2]
min_inv=100

# Creates a list of production cost

prod_cost_1 =[ [0,13.00, 10.65, 9.60],\
            [ 17.40,14.10,11.20,9.45],\
            [ 17.40,14.22,11.00,9.50],\
            [0,14.30,11.25,9.60],\
            [ 17.50, 13.80,11.40,9.60],\
            [ 18.25, 13.90,11.40, 8.90],\
            [ 19.75,13.90,10.75,9.40 ]]
prod_cost_1= makeDict([factory,yarn],prod_cost_1)
prod_cost_2 =[ [0,13.00, 10.65, 9.60],\
            [ 17.40,14.10,11.20,9.45],\
            [ 17.40,14.22,11.00,9.50],\
            [0,14.30,11.25,9.60],\
            [ 17.50, 13.80,11.40,9.60],\
            [ 18.25, 13.90,11.40, 8.90],\
            [ 19.75,13.90,10.75,9.40 ]]
prod_cost_2= makeDict([factory,yarn],prod_cost_2)
prod_cost={1:prod_cost_1,2:prod_cost_2}


# Creates a list of transportation cost
trans_cost_1 = [
    [ 0.30,0.30,0.45, 0.45],\
    [ 0.40,0.40,0.60,0.60],\
    [ 0.80,0.80, 1.20,1.20],\
    [ 0.70,0.70,1.05,1.05],\
    [ 0.70, 0.70, 1.05, 1.05],\
    [ 0,0,0,0],\
    [ 0.50, 0.50, 0.75, 0.75]]
trans_cost_1= makeDict([factory,yarn],trans_cost_1)
trans_cost_2 = [
    [ 0.30,0.30,0.45, 0.45],\
    [ 0.40,0.40,0.60,0.60],\
    [ 0.80,0.80, 1.20,1.20],\
    [ 0.70,0.70,1.05,1.05],\
    [ 0.70, 0.70, 1.05, 1.05],\
    [ 0,0,0,0],\
    [ 0.50, 0.50, 0.75, 0.75]]

trans_cost_2= makeDict([factory,yarn],trans_cost_2)
trans_cost={1:trans_cost_1,2:trans_cost_2}

# Creates a list of processing time
proc_time_1=[
    [0, 0.400,0.375,0.250],\
    [0.700,0.500,0.350,0.250],\
    [0.675,0.450,0.400,0.250],\
    [0,0.450,0.350,0.200],\
    [0.650,0.450,0.400,0.250],\
    [0.625,0.500,0.425,0.425],\
    [0.700,0.450,0.350,0.400],\
]
proc_time_1= makeDict([factory,yarn],proc_time_1)
proc_time_2=[
    [0, 0.400,0.375,0.250],\
    [0.700,0.500,0.350,0.250],\
    [0.675,0.450,0.400,0.250],\
    [0,0.450,0.350,0.200],\
    [0.650,0.450,0.400,0.250],\
    [0.625,0.500,0.425,0.425],\
    [0.700,0.450,0.350,0.400],\
]
proc_time_2= makeDict([factory,yarn],proc_time_2)
proc_time={1:proc_time_1,2:proc_time_2}

capacity = {1:{
    "A":2500,
"B":3000,
"C":2500,
"D":2600,
"E":2500,
"F":38000,
"G":2500},2:{
    "A":2500,
"B":3000,
"C":2500,
"D":2600,
"E":2500,
"F":38000,
"G":2500}}
demand={1:{
    "1":25000,
    "2":26000,
    "3":28000,
    "4":28000},2:{
    "1":25000,
    "2":26000,
    "3":28000,
    "4":28000}
}

# holding cost of invetory
h={
    "1":1.3,
    "2":1.8,
    "3":1.1,
    "4":0.9}
# decision variables
production_vars = LpVariable.dicts("x", (T,factory,yarn),lowBound=0, cat='Continuous')
inv_vars=LpVariable.dicts("inv", (T,yarn),lowBound=0, cat='Continuous')
#initial inventory
inv0={
    "1":0,
    "2":0,
    "3":0,
    "4":0}
demand[1]["1"]

25000

In [3]:
#objective function
probA+=lpSum([h[j]*inv_vars[t][j] for t in T[1:] for j in yarn])\
+lpSum([production_vars[t][i][j]*(prod_cost[t][i][j]+trans_cost[t][i][j]) for t in T[1:] \
        for i in factory for j in yarn]),"total cost"
for j in yarn:
    probA+=inv_vars[0][j]==inv0[j],"initilization %s"%j
# demand constraint
for t in T[1:]:
    #demand constraints
    for j in yarn:
        probA += inv_vars[t-1][j]+lpSum([production_vars[t][i][j] for i in factory ]) - demand[t][j] ==inv_vars[t][j],\
        "demand_%d%s"%(t,j)
        # minimum inventory constraint
        probA += inv_vars[t][j]>= min_inv,"min_inv_%d%s"%(t,j)
    #capacity constraint
    
    for i in factory:
        probA += lpSum([proc_time[t][i][j]*production_vars[t][i][j] for j in yarn]) <= capacity[t][i],\
        "capacity_%d%s"%(t,i)
   
    # x_1A and x_1D constraint
    probA += production_vars[t]["A"]["1"]==0,"xA1_%d"%t
    probA += production_vars[t]["D"]["1"]==0,"xD1_%d"%t
    

   

In [4]:
probA.writeLP("Filatoi_multiperiod.lp")
probA.solve()
print("Status:",LpStatus[probA.status])
#print optimization result
for v in probA.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
print("Total cost =", value(probA.objective))
original_cost=value(probA.objective)

print("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
for name, c in list(probA.constraints.items()):
    print(name, ":", c, "\t", c.pi, "\t\t", c.slack)

Status: Optimal
inv_0_1 = 0.0 	Reduced Cost = 0.0
inv_0_2 = 0.0 	Reduced Cost = 0.0
inv_0_3 = 0.0 	Reduced Cost = 0.0
inv_0_4 = 0.0 	Reduced Cost = 0.0
inv_1_1 = 100.0 	Reduced Cost = 0.0
inv_1_2 = 100.0 	Reduced Cost = 0.0
inv_1_3 = 100.0 	Reduced Cost = 0.0
inv_1_4 = 100.0 	Reduced Cost = 0.0
inv_2_1 = 100.0 	Reduced Cost = 0.0
inv_2_2 = 100.0 	Reduced Cost = 0.0
inv_2_3 = 100.0 	Reduced Cost = 0.0
inv_2_4 = 100.0 	Reduced Cost = 0.0
x_1_A_1 = 0.0 	Reduced Cost = 0.0
x_1_A_2 = 6250.0 	Reduced Cost = 0.0
x_1_A_3 = 0.0 	Reduced Cost = 0.35514706
x_1_A_4 = 0.0 	Reduced Cost = 1.2867647
x_1_B_1 = 4285.7143 	Reduced Cost = 0.0
x_1_B_2 = 0.0 	Reduced Cost = 0.80798319
x_1_B_3 = 0.0 	Reduced Cost = 0.38676471
x_1_B_4 = 0.0 	Reduced Cost = 0.88340336
x_1_C_1 = 3703.7037 	Reduced Cost = 0.0
x_1_C_2 = 0.0 	Reduced Cost = 0.97686275
x_1_C_3 = 0.0 	Reduced Cost = 0.71394336
x_1_C_4 = 0.0 	Reduced Cost = 1.4087146
x_1_D_1 = 0.0 	Reduced Cost = 0.0
x_1_D_2 = 0.0 	Reduced Cost = 0.041176471
x_1_D_3